In [1]:
!git clone https://github.com/mlfoundations/rtfm.git

Cloning into 'rtfm'...
remote: Enumerating objects: 825, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 825 (delta 148), reused 155 (delta 129), pack-reused 633 (from 1)
Receiving objects: 100% (825/825), 19.12 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (525/525), done.


In [4]:
import sys
sys.path.append('/content/rtfm')  # Update this path

In [3]:
from google.colab import files
uploaded = files.upload()

Saving requirements.txt to requirements.txt


In [4]:
! pip install -r '/content/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 14.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of fairlearn to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
!pip install git+https://github.com/jpgard/llama-recipes.git
!pip install -e .
!pip install --no-deps git+https://github.com/mlfoundations/tableshift.git
!pip install git+https://github.com/mlfoundations/tabliblib.git

  Cloning https://github.com/jpgard/llama-recipes.git to /tmp/pip-req-build-_82o6cwu
  Running command git clone --filter=blob:none --quiet https://github.com/jpgard/llama-recipes.git /tmp/pip-req-build-_82o6cwu
  Resolved https://github.com/jpgard/llama-recipes.git to commit 186213f2ba3382f0aaa29326377e36c840d78160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-recipes: filename=llama_recipes-0.0.1-py3-none-any.whl size=64196 sha256=bb234e4223b39154d1bad8bfcdc1630d9ba7577713dbafe483d4a766cbbc8779
  Stored in directory: /tmp/pip-ephem-wheel-cache-k0f_dukg/wheels/aa/8c/49/845e5dd43656becf00798c933b2187faf09cb977c931bb69d3
Successfully built llama-recipes
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  Cloning https://github.com/mlfoundations/tableshift.git to /tmp/pip-req-build-_uaq2g_v

In [2]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
import pandas as pd
import torch
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig

from rtfm.configs import SerializerConfig, TrainConfig, TokenizerConfig
from rtfm.inference_utils import InferenceModel
from rtfm.serialization.serializers import get_serializer
from rtfm.tokenization.text import prepare_tokenizer

train_config = TrainConfig(model_name="mlfoundations/tabula-8b", context_length=8192)

# If using a base llama model (not fine-tuned TabuLa),
# make sure to set add_serializer_tokens=False
# (because we do not want to use special tokens for
# the base model which is not trained on them).
tokenizer_config = TokenizerConfig()

# Load the configuration
config = AutoConfig.from_pretrained(train_config.model_name)

# Set the torch_dtype to bfloat16 which matches TabuLa train/eval setup
config.torch_dtype = 'bfloat16'

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

model = LlamaForCausalLM.from_pretrained(
    train_config.model_name, device_map="cuda", config=config).to(device)

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
serializer = get_serializer(SerializerConfig())

tokenizer, model = prepare_tokenizer(
    model,
    tokenizer=tokenizer,
    pretrained_model_name_or_path=train_config.model_name,
    model_max_length=train_config.context_length,
    use_fast_tokenizer=tokenizer_config.use_fast_tokenizer,
    serializer_tokens_embed_fn=tokenizer_config.serializer_tokens_embed_fn,
    serializer_tokens=serializer.special_tokens
    if tokenizer_config.add_serializer_tokens
    else None,
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)


cuda


model-00001-of-00007.safetensors:  14%|#4        | 703M/4.89G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
labeled_examples = pd.DataFrame(
    [
        {"location": "New York", "temperature": 22, "humidity": 65, "wind_speed": 12, "pressure": 1012, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Los Angeles", "temperature": 26, "humidity": 60, "wind_speed": 7, "pressure": 1015,
         "month": "July", "weather_yesterday": "Partly Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Chicago", "temperature": 18, "humidity": 70, "wind_speed": 15, "pressure": 1008, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.1, "visibility": 8, "weather_today": "Cloudy"},
        {"location": "Houston", "temperature": 30, "humidity": 80, "wind_speed": 10, "pressure": 1010, "month": "July",
         "weather_yesterday": "Rain", "precipitation": 0.5, "visibility": 7, "weather_today": "Rain"},
        {"location": "Phoenix", "temperature": 35, "humidity": 20, "wind_speed": 5, "pressure": 1005, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
        {"location": "Philadelphia", "temperature": 24, "humidity": 75, "wind_speed": 14, "pressure": 1009,
         "month": "July", "weather_yesterday": "Partly Cloudy", "precipitation": 0.2, "visibility": 9,
         "weather_today": "Partly Cloudy"},
        {"location": "San Antonio", "temperature": 28, "humidity": 68, "wind_speed": 11, "pressure": 1011,
         "month": "July", "weather_yesterday": "Rain", "precipitation": 0.4, "visibility": 8, "weather_today": "Rain"},
        {"location": "San Diego", "temperature": 22, "humidity": 65, "wind_speed": 10, "pressure": 1014,
         "month": "July", "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Partly Sunny"},
        {"location": "Dallas", "temperature": 27, "humidity": 72, "wind_speed": 9, "pressure": 1007, "month": "July",
         "weather_yesterday": "Partly Cloudy", "precipitation": 0.3, "visibility": 9, "weather_today": "Cloudy"},
    ]
)
target_example = pd.DataFrame(
    [
        {"location": "San Jose", "temperature": 23, "humidity": 55, "wind_speed": 8, "pressure": 1013, "month": "July",
         "weather_yesterday": "Sunny", "precipitation": 0, "visibility": 10, "weather_today": "Sunny"},
    ]
)

output = inference_model.predict(
    target_example=target_example,
    target_colname="weather_today",
    target_choices=["Sunny", "Partly Sunny", "Cloudy", "Partly Cloudy", "Rain"],
    labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction for sample 
    location  temperature  humidity  wind_speed  pressure month  \
0  San Jose           23        55           8      1013  July   

  weather_yesterday  precipitation  visibility weather_today  
0             Sunny              0          10         Sunny   
 is: Sunny


In [11]:
from rtfm.serialization.serialization_utils import discretize_continuous_column

examples = pd.DataFrame(
    [
    {"location": "New York", "size_sqft": 1200, "bedrooms": 3, "bathrooms": 2, "age": 10, "lot_size_acres": 0.15, "garage": True, "price": 850},
    {"location": "Los Angeles", "size_sqft": 1500, "bedrooms": 4, "bathrooms": 3, "age": 8, "lot_size_acres": 0.25, "garage": True, "price": 950},
    {"location": "Chicago", "size_sqft": 1300, "bedrooms": 3, "bathrooms": 2, "age": 15, "lot_size_acres": 0.2, "garage": False, "price": 700},
    {"location": "Houston", "size_sqft": 1700, "bedrooms": 4, "bathrooms": 3, "age": 5, "lot_size_acres": 0.3, "garage": True, "price": 650},
    {"location": "Phoenix", "size_sqft": 1600, "bedrooms": 3, "bathrooms": 2, "age": 7, "lot_size_acres": 0.25, "garage": True, "price": 750},
    {"location": "Philadelphia", "size_sqft": 1400, "bedrooms": 3, "bathrooms": 2, "age": 12, "lot_size_acres": 0.18, "garage": False, "price": 600},
    {"location": "San Antonio", "size_sqft": 1800, "bedrooms": 4, "bathrooms": 3, "age": 3, "lot_size_acres": 0.4, "garage": True, "price": 700},
    {"location": "San Diego", "size_sqft": 1550, "bedrooms": 3, "bathrooms": 2, "age": 9, "lot_size_acres": 0.22, "garage": True, "price": 850},
    {"location": "Dallas", "size_sqft": 1450, "bedrooms": 3, "bathrooms": 2, "age": 11, "lot_size_acres": 0.19, "garage": True, "price": 700},
    {"location": "San Jose", "size_sqft": 1600, "bedrooms": 4, "bathrooms": 3, "age": 6, "lot_size_acres": 0.2, "garage": False, "price": 800},
    {"location": "Seattle", "size_sqft": 1800, "bedrooms": 4, "bathrooms": 2, "age": 10, "lot_size_acres": 0.2, "garage": False, "price": 925},
]
)

examples["price"] = discretize_continuous_column(examples["price"], num_buckets=4)
target_choices = examples["price"].unique().tolist()

target_example = examples.iloc[[0]]
labeled_examples = examples.iloc[1:]


output = inference_model.predict(
    target_example=target_example,
    target_colname="price",
    target_choices=target_choices,
    labeled_examples=labeled_examples,
)
print(f"Prediction for sample \n {target_example} \n is: {output}")

/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction for sample 
    location  size_sqft  bedrooms  bathrooms  age  lot_size_acres  garage  \
0  New York       1200         3          2   10            0.15    True   

                price  
0  greater than 850.0   
 is: greater than 850.0
